# fct_operator_hourly_summary

In [18]:
import datetime
import sys

import _operator_grain_route_dir_visuals as _report_route_dir_visuals
import _operator_grain_scheduled_service 
import _sql_query
import google.auth
import pandas as pd
from google.cloud import bigquery
from IPython.display import HTML, Image, Markdown, display, display_html
from loguru import logger
from omegaconf import OmegaConf
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SEGMENT_GCS

readable_dict = OmegaConf.load("readable2.yml")

import altair as alt
import geopandas as gpd
from shapely import wkt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_name = "City and County of San Francisco"

In [4]:
df = _sql_query.download_with_pandas_gbq(
    project="cal-itp-data-infra-staging",
    filename="tiffany_mart_gtfs_rollup.fct_operator_hourly_summary",
)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_operator_hourly_summary`
        WHERE month_first_day >=  DATE('2025-01-01')
    


/opt/conda/lib/python3.11/site-packages/pandas_gbq/gbq.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa


Downloading: 100%|██████████|
download time: 0:00:11.291701


In [8]:
sf_only = df.loc[df.analysis_name == analysis_name]

In [9]:
sf_only.month_first_day.unique()

array(['2025-08-01T00:00:00.000000000', '2025-10-01T00:00:00.000000000',
       '2025-09-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [10]:
sf_only.columns = sf_only.columns.str.replace("_", " ").str.title()

In [12]:
saturday = sf_only.loc[sf_only["Day Type"] == "Saturday"]

In [14]:
saturday = saturday.rename(columns = {"Month First Day":"Date"})

In [36]:
saturday

,Date,Name,Analysis Name,Day Type,Departure Hour,N Trips
5016,2025-09-01,Bay Area 511 Muni Schedule,City and County of San Francisco,Saturday,30,5
5017,2025-10-01,Bay Area 511 Muni Schedule,City and County of San Francisco,Saturday,30,5
6405,2025-09-01,Bay Area 511 Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,9,7
6409,2025-08-01,Bay Area 511 Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,9,7
6410,2025-10-01,Bay Area 511 Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,9,7
6614,2025-10-01,Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,9,7
6615,2025-08-01,Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,9,7
6617,2025-09-01,Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,9,7
7073,2025-09-01,Bay Area 511 Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,17,8
7075,2025-08-01,Bay Area 511 Golden Gate Park Shuttle Schedule,City and County of San Francisco,Saturday,10,8


In [15]:
selection = alt.selection_point(fields=["Date"], bind="legend")

In [34]:
def line_chart(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    tooltip_cols: list,
    date_format: str = "%b %Y",
):
    # Set chart
    chart = (
        alt.Chart(df)
        .mark_line(size=3)
        .encode(
            x=alt.X(
                x_col,
                title=x_col,
                axis=alt.Axis(labelAngle=-45, format=date_format),
            ),
            y=alt.Y(
                f"{y_col}:Q",
                title=y_col,
            ),
            tooltip=tooltip_cols,
        )
    )
    return chart

In [35]:
line_chart(
    df=saturday,
    x_col="Date",
    y_col="Departure Hour",
    tooltip_cols=[
        "Date",
        "Departure Hour",
    ],
)

alt.Chart(...)